In [ ]:
import sys

# if ra2ce is not found sys.path.append(r'C:path_to_the_ra2ce_repo')  

import geopandas as gpd
import pickle

import pyproj
from networkx import MultiDiGraph
from pathlib import Path
from snkit import Network

from ra2ce.graph.exporters.multi_graph_network_exporter import MultiGraphNetworkExporter
from ra2ce.graph.exporters.network_exporter_base import NetworkExporterBase
from ra2ce.graph.network_config_data.network_config_data import NetworkConfigData, NetworkSection
from ra2ce.graph.network_wrappers.osm_network_wrapper.osm_network_wrapper import OsmNetworkWrapper
from ra2ce.graph.origins_destinations import add_od_nodes
from ra2ce_multi_network.simplify_rail import _nx_to_network, _network_to_nx
import ra2ce.graph.networks_utils as nut

In [ ]:
# Defining ini variables
root_folder = Path(
    r'path_to_od_file')
od_file = root_folder.joinpath('od_nodes.geojson')

In [ ]:
# Load complex graph
od_gdf = gpd.read_file(od_file)

with open(root_folder.joinpath(f'road_complex_graph.p'), 'rb') as f:
    complex_graph = pickle.load(f)

In [ ]:
# Simplify complex graph => get_network procedure source == "OSM download"

simple_graph, complex_graph, link_tables = nut.create_simplified_graph(complex_graph)

In [ ]:
# Save the link tables linking complex and simple IDs
osm_network_wrapper = OsmNetworkWrapper(config_data=NetworkConfigData())
# osm_network_wrapper._export_linking_tables(link_tables)

if not osm_network_wrapper.is_directed and isinstance(simple_graph, MultiDiGraph):
    simple_graph = simple_graph.to_undirected()

# Check if all geometries between nodes are there, if not, add them as a straight line.

simple_graph = nut.add_missing_geoms_graph(simple_graph, geom_name="geometry")
simple_graph.graph["crs"] = pyproj.CRS("EPSG:4326")

In [ ]:
# Save the outputs

od_gdf.to_file(od_file, driver="GeoJSON")

simple_graph_exporter = MultiGraphNetworkExporter(
    basename=f'road_simple_graph',
    export_types=['pickle', 'shp'])
simple_graph_exporter.export_to_shp(output_dir=root_folder.joinpath(f'output_graph'),
                                    export_data=simple_graph)
simple_graph_exporter.export_to_pickle(output_dir=root_folder.joinpath(f'output_graph'),
                                       export_data=simple_graph)

In [ ]:
# Load graphs and networks prepared befor

od_gdf = gpd.read_file(od_file)

# Approach 1
with open(root_folder.joinpath(f'road_simple_graph.p'), 'rb') as f:
    simple_graph = pickle.load(f)

simple_road_network = Network(
        nodes=_nx_to_network(simple_graph).nodes,
        edges=_nx_to_network(simple_graph).edges
    )

# Approach 2
# simple_road_network = Network(
#     nodes=gpd.read_file(root_folder.joinpath(f'road_simple_graph_nodes.gpkg')),
#     edges=gpd.read_file(root_folder.joinpath(f'road_simple_graph_edges.gpkg'))
# )

In [ ]:
# Map origin-destinations the simple graphs

road_simple_graph_od_mapped = add_od_nodes(od=gpd.read_file(od_file), graph=simple_graph, crs=pyproj.CRS("EPSG:4326"))[1]

In [ ]:
# Save the outputs

od_gdf.to_file(od_file, driver="GeoJSON")

simple_graph_od_mapped_exporter = MultiGraphNetworkExporter(
    basename=f'road_simple_graph_od_mapped',
    export_types=['pickle', 'shp'])
simple_graph_od_mapped_exporter.export_to_shp(output_dir=root_folder.joinpath(f'output_graph'),
                                    export_data=road_simple_graph_od_mapped)
simple_graph_od_mapped_exporter.export_to_pickle(output_dir=root_folder.joinpath(f'output_graph'),
                                       export_data=road_simple_graph_od_mapped)

In [ ]:
# Visualise

road_map = road_simple_graph_od_mapped.edges.explore(tiles="CartoDB positron")
road_map = od_gdf.explore(m=road_map, color='springgreen')
road_simple_graph_od_mapped.nodes[~road_simple_graph_od_mapped.nodes['od_id'].isna()].explore(m=road_map, color='darkgreen')